In [13]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import TextLoader

loader = TextLoader("file.txt")
loader.load()



AIMessage(content="In the not-too-distant future, a groundbreaking invention emerged within the bustling city of Veridium. It was the Emotisphere, a revolutionary device that allowed emotions to be bought and sold as commodities. With a simple implant, individuals could experience any emotion they desired, simply by activating the corresponding program. The world was eager to embrace this new technology, believing it would bring happiness, eliminate suffering, and revolutionize the way society functioned.\n\nAs the Emotispheres were distributed, people's lives changed dramatically. Relationships became strained, as genuine emotional connections became a rarity. Love, once a profound and heartfelt experience, was reduced to a mere transaction. People traded their love for fleeting moments of passion, and the concept of commitment vanished. The deeper meaning of relationships waned, and many lost their ability to truly connect with others.\n\nSociety transformed into a cold and detached 

ImportError: PyMuPDF package not found, please install it with `pip install pymupdf`